# 1 dimensional laws

In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import tensorflow as tf

2022-03-09 17:09:57.316820: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 17:09:57.316850: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Wolfram's 256 Rules

These are all cellular automaton rules for a 1-d array with only neares neighbour interaction. There are therefore only three cells (L,C,R) determining the outcome of one cell (C). This gives 2^3=8 possible combinations that can give rise to an outcome of 2 possibilities. Thus there are 2^8=256 possible rules. 

The following function generates each of these rules.
 
Input:  
num = number of rule  
inp = string of input bits   
t = number of timesteps calculated  

Output: an array of arrays each for one timestep beginning with the input array. 

The boudary conditions are periodic
        

In [2]:
def CellularAutomata(num,inp,t):
    bnum = np.flip(np.fromiter(np.binary_repr(num,width=8),dtype=int))
    out = [np.fromiter(inp,dtype=int)]
    lin = len(inp)
    currinp = inp
    for n in range(t):
        outnow = []
        for i in range(len(inp)):
            outnow = np.append(outnow,bnum[int(str(currinp[(i-1)%lin])+str(currinp[(i)%lin])+str(currinp[(i+1)%lin]),2)])

        currinp =''.join([str(int(elem)) for elem in outnow])
        out = np.append(out,[outnow],axis=0)
    return(out)


## Generating Data

In [103]:
N=100 #batch size
L=3 #Length of box
rule= 170
time = 1

x_train=[]
y_train=[]


for i in range(N):
    #x=[0,0,0]
    #x[np.random.randint(0,3)]=1
    x=np.random.randint(0,2,L)
    x_train.append(np.copy(x))
    #y_train.append(np.copy(CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time]))
    y_train.append(np.copy(CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time,1]))
    #print(''.join([str(int(elem)) for elem in x]),CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time,0])

#for i in range(20):
 #   print(x_train[i],y_train[i])
    
x_train = np.array(x_train).reshape(N,L,1)
y_train = np.array(y_train).reshape(N,1)
#print(x_train.shape,y_train.shape)
#print(x_train[0],y_train[0],CellularAutomata(rule,''.join([str(int(elem)) for elem in x_train[0]]),time)[time])

## Models
useful resource for understanding what is going on:  
Padding https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/  
Guide to 2d conv layers https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/  
Guide to 1d conv layers https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/  
shapes, dimensions and units https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc  


### Dense layers

### 1-D Convolutional layer

Tutorial customb layers: https://www.tensorflow.org/tutorials/customization/custom_layers

In [128]:
##### Simple convolution without stride ######
class SimpleConv(tf.keras.layers.Layer):

    def __init__(self):
        super(SimpleConv, self).__init__()

    def build(self, input_shape):
        self.shape = input_shape
        self.w = self.add_weight(shape=(input_shape[-1], 1),
                               initializer='random_normal',
                               trainable=True)
        print(input_shape[-1])
    def call(self, inputs):
        print(self.w)
        return tf.add(np.array(tf.math.multiply(inputs, tf.transpose(self.w))).reshape(self.shape[0],self.shape[1],1),0)

In [138]:
layer = SimpleConv()
dense = tf.keras.layers.Dense(1)
layer(np.array([[1.,1,1],[1,2,3],[0,0,0],[0,0,0]]))
dense(layer(np.array([[1.,1,1],[1,2,3],[0,0,0],[0,0,0]])))

3
<tf.Variable 'simple_conv_93/Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.03635658],
       [ 0.03677227],
       [-0.04227185]], dtype=float32)>
<tf.Variable 'simple_conv_93/Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.03635658],
       [ 0.03677227],
       [-0.04227185]], dtype=float32)>


<tf.Tensor: shape=(4, 3, 1), dtype=float32, numpy=
array([[[ 0.01720952],
        [ 0.01740629],
        [-0.02000954]],

       [[ 0.01720952],
        [ 0.03481259],
        [-0.06002861]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ]]], dtype=float32)>

In [132]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer((N,3))) 
model.add(SimpleConv())
model.add(SimpleConv())
model.add(SimpleConv())
model.add(SimpleConv())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3,activation="softmax"))
model.add(tf.keras.layers.Dense(1,activation="softmax"))
model.summary()


3
<tf.Variable 'simple_conv_87/Variable:0' shape=(3, 1) dtype=float32>


NotImplementedError: Exception encountered when calling layer "simple_conv_87" (type SimpleConv).

in user code:

    File "/tmp/ipykernel_21509/59490006.py", line 15, in call  *
        return tf.add(np.array(tf.math.multiply(inputs, tf.transpose(self.w))).reshape(self.shape[0],self.shape[1],1),0)

    NotImplementedError: Cannot convert a symbolic Tensor (simple_conv_87/Mul:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported


Call arguments received:
  • inputs=tf.Tensor(shape=(None, 100, 3), dtype=float32)

## Execute learning

In [104]:
#loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
#loss_fn = tf.keras.metrics.BinaryAccuracy()
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
#loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="auto",name="sparse_categorical_crossentropy")


predictions = model(x_train).numpy()

loss_fn(y_train, predictions).numpy()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1),
              loss=loss_fn,
              metrics=[tf.keras.metrics.BinaryAccuracy()]
                #metrics= [tf.keras.losses.CategoricalCrossentropy()]
             )
train_history = model.fit(x=x_train, y=y_train, epochs=5,verbose=1)
plt.title("Loss")
plt.plot(train_history.history['loss'], 'k')

ValueError: Input 0 of layer "sequential_13" is incompatible with the layer: expected shape=(None, 100, 3), found shape=(100, 3, 1)